In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Path to your chromedriver.exe
chromedriver_path = r"C:\Users\merye\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# Create a Service object with the path to chromedriver
service = Service(chromedriver_path)
#service1 pour chaque add (on ne peut pas utiliser le meme  service car le quit du driver1 engendre le quit du driver principal)
service1 = Service(chromedriver_path)

# Fonction pour créer une nouvelle instance 
def create_new_driver(service):
    driver = webdriver.Chrome(service=service)
    wait = WebDriverWait(driver, 10)
    return driver, wait


driver = webdriver.Chrome(service=service)

# Modifier le gouvernement chaque fois 
City = "Monastir"
base_url = 'https://www.tayara.tn/ads/c/Immobilier/l/'+City+'/?page='


houses_data = []

# Wait for page load and elements to be visible
wait = WebDriverWait(driver, 10)


for page_num in range(1, 15):  
    driver.get(base_url + str(page_num))

    # Wait until the article elements are present on the page
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//article")))

    # Loop through the articles on the page
    house_cards = driver.find_elements(By.XPATH, "//article")
    print(f"Page {page_num}: Found {len(house_cards)} houses.")

    # Extract details from each house card
    for card in house_cards:
        try:
            # Extraire le lien de chaque maison pour ouvrir une nouvelle page (href)
            link = card.find_element(By.XPATH, ".//a").get_attribute("href").strip()
        except:
            link = "N/A"

        try:
            #  Titre 
            title = card.find_element(By.XPATH, ".//h2").text.strip()
        except:
            title = "N/A"

        try:
            # Prix
            price = card.find_element(By.XPATH, ".//data").get_attribute("value").strip()
        except:
            price = "N/A"

        try:
            #  Category
            category = card.find_element(By.XPATH, ".//div[contains(@class, 'text-gray-800')]").text.strip()
        except:
            category = "N/A"

        if link != "N/A":
            print(f"Navigating to: {link}")
            try:
                # Create a new WebDriver instance for each link
                driver1, wait1 = create_new_driver(service1)
                # Navigate to the detailed page
                driver1.get(link)

                # Waiting for the new page to load
                wait1.until(EC.presence_of_element_located((By.XPATH, "//h1")))

                try:
                    # Transaction Type (à vendre or à louer)
                    transaction_type = driver1.find_element(By.XPATH, ".//span[contains(text(),'Type de transaction')]/following-sibling::span").text.strip()
                except:
                    transaction_type = "N/A"

                try:
                    # Superficie
                    surface_area = driver1.find_element(By.XPATH, ".//span[contains(text(),'Superficie')]/following-sibling::span").text.strip()
                except:
                    surface_area = "N/A"

                try:
                    # Nombre Salles de bains
                    num_bathrooms = driver1.find_element(By.XPATH, ".//span[contains(text(),'Salles de bains')]/following-sibling::span").text.strip()
                except:
                    num_bathrooms = "N/A"

                try:
                    # Nombre Chambres
                    num_bedrooms = driver1.find_element(By.XPATH, ".//span[contains(text(),'Chambres')]/following-sibling::span").text.strip()
                except:
                    num_bedrooms = "N/A"

                
                houses_data.append({
                    "link": link,
                    "title": title,
                    "price": price,
                    "category": category,
                    "transaction_type": transaction_type,
                    "surface_area": surface_area,
                    "num_bathrooms": num_bathrooms,
                    "num_bedrooms": num_bedrooms,
                    "gouvernement": City,
                })
            except Exception as e:
                print(f"Error navigating to {link}: {e}")
            finally:
                
                driver1.quit()


df = pd.DataFrame(houses_data)
print(df)
driver.quit()
df.to_csv(City+'.csv', index=False)

Page 1: Found 34 houses.
Navigating to: https://www.tayara.tn/item/674a218d3c20a99fb55bf6d8/Terrains%20et%20Fermes/Monastir/Sayada_Lamta_Bou_Hajar/Terrain_Vendre_Bouhjar_Monastir_/
Navigating to: https://www.tayara.tn/item/671781d71cde9a414fada890/Autres%20Immobiliers/Monastir/Sahline/Dpt_louer_/
Navigating to: https://www.tayara.tn/item/6756bb408729cead59e87d34/Locations%20de%20vacances/Monastir/Monastir/A_Louer_pour_les_vacances_/
Navigating to: https://www.tayara.tn/item/6737165a0d8480a1e00edf38/Locations%20de%20vacances/Monastir/Monastir/A_Louer_pour_les_vacances_/
Navigating to: https://www.tayara.tn/item/674a218d3c20a99fb55bf6d8/Terrains%20et%20Fermes/Monastir/Sayada_Lamta_Bou_Hajar/Terrain_Vendre_Bouhjar_Monastir_/
Navigating to: https://www.tayara.tn/item/675da559f39faeaa98b96967/Terrains%20et%20Fermes/Monastir/Jemmal/Terrain_centre_ville_Jammel/
Navigating to: https://www.tayara.tn/item/673caaf10d8480a1e0124d67/Magasins%2C%20Commerces%20et%20Locaux%20industriels/Monastir/Monas

In [ ]:
import pandas as pd

# Liste des gouvernements en Tunisie
Govs = [
    "Ariana", "Beja", "Ben-arous", "Bizerte", "Gabes", "gafsa", "jendouba", 
    "Kairouan", "Kasserine", "Kebeli", "Manouba", "Kef", "Mahdia", "Medenine", 
    "Monastir", "Sidi-bouzid", "Siliana", "Sousse", "Tataouine", "Tozeur", 
    "Zaghouan", "Sfax", "Nabeul", "Tunis"
]


full_data = pd.concat([pd.read_csv(f"./{g}.csv") for g in Govs])

# Remove the index column
full_data = full_data.drop(["Unnamed: 0"], axis=1)


full_data.to_csv("./Scraped-data-tayara.tn.csv", index=False)
